In [1]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
%matplotlib inline
style.use('bmh')

import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np


Предобработка из прошлого

In [3]:
df = pd.read_csv('bank.csv', delimiter = ';', na_values = 'unknown')
df['default'].fillna('unknown', inplace = True)
df.drop('pdays',axis = 1, inplace = True)

df['y'].replace({'yes':1, 'no':0}, inplace = True)

data_types = df.dtypes
int_cols = [df.columns[i] for i in range(df.shape[1]) if data_types[i] == 'int64']
float_cols = [df.columns[i] for i in range(df.shape[1]) if data_types[i] == 'float64']
cat_cols = [df.columns[i] for i in range(df.shape[1]) if data_types[i] == 'O']

mon_num = {'mar':1, 'apr':2, 'may':3, 'jun': 4, 'jul':5, 'aug':6, 'nov':7, 'sep':7, 'oct':8, 'nov':9, 'dec':10}
day_num = {day:i for day, i in zip(df['day_of_week'].unique(), range(1, 6))}
education_sorted = {'illiterate': 0, 'unknown': 1, 'basic.4y': 2,
                    'basic.6y': 3, 'basic.9y': 4, 'high.school': 5,
                    'professional.course': 6, 'university.degree': 7}

ord_mon = OrdinalEncoder(mapping=[{'col': 'month', 'mapping': mon_num}])
ord_day =  OrdinalEncoder(mapping=[{'col': 'day_of_week', 'mapping': day_num}])
ord_edu = OrdinalEncoder(mapping=[{'col': 'education', 'mapping': education_sorted}])

cat_steps = [('encoder', BinaryEncoder(drop_invariant = True)),
            ('imputer', KNNImputer(weights = 'distance')),
              ('scaler', StandardScaler())
             ]


transf = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)

transformer = ColumnTransformer(
    [
    ('encode_scale', Pipeline(steps = cat_steps), list(set(cat_cols)-set(['day_of_week', 'month', 'y']))),
    ('month', Pipeline(steps = [('ord_mon', ord_mon) , ('scale_mon', StandardScaler())]), 'month'),
    ('day', Pipeline(steps = [('ord_day', ord_day), ('scale_day', StandardScaler())]), 'day_of_week'),
    ('scale', StandardScaler(), ['age', 'duration', *float_cols, 'previous', 'campaign'])
    ]
)
    

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df, df['y'], test_size = 0.2, random_state = 69)

X_train = transformer.fit_transform(X_train);
X_test = transformer.transform(X_test);

C:\Users\Eugene\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\Eugene\anaconda3\lib\site-packages\category_encoders\basen.py:236: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_out.drop(col, 1, inplace=True)
C:\Users\Eugene\anaconda3\lib\site-packages\category_encoders\basen.py:236: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_out.drop(col, 1, inplace=True)


Опять же возьмем удивительную функцию из прошлой домашки и переберем сразу все.

In [5]:
def search_best_model(X, y, model, param_grid, param_scale='ordinary', draw=True, multi = False, scoring = 'roc_auc'):
    
    CV_model = GridSearchCV(estimator=model, 
                            param_grid=param_grid,
                            cv=3, 
                            scoring= scoring,
                            n_jobs=1, 
                            verbose=1,
                           refit = False)
    CV_model.fit(X, y)

    return CV_model

In [23]:
models = [KNeighborsClassifier(),
          DecisionTreeClassifier(),
          LogisticRegression(),
          SVC(kernel = 'poly')]
param_grids = [{'n_neighbors':np.array(np.linspace(4, 30, 8), dtype='int')},
               {'max_depth': np.arange(1, 30)},
               {'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]},
                {'C':[0.001, 0.01, 0.1, 1, 10],
                'degree': [2, 3, 4]}
              ]


In [24]:
res = []
for model, param_grid in zip(models, param_grids):
    res.append(search_and_draw(X_train, y_train, model, param_grid, scoring = 'f1', draw = False))

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 29 candidates, totalling 87 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Fitting 3 folds for each of 15 candidates, totalling 45 fits


In [36]:
best_params = [model.best_params_ for model in res]
best_score_test = [model.best_score_ for model in res]
best_params

[{'n_neighbors': 7}, {'max_depth': 3}, {'C': 0.1}, {'C': 10, 'degree': 3}]

In [44]:
knn = KNeighborsClassifier(n_neighbors = 7)
tree = DecisionTreeClassifier(max_depth = 3)
lr = LogisticRegression(C = 0.1)
svm = SVC(kernel = 'poly', C = 10, degree = 3)

knn.fit(X_train, y_train)
tree.fit(X_train, y_train)
lr.fit(X_train, y_train)
svm.fit(X_train, y_train)


SVC(C=10, kernel='poly')

In [45]:
knn_y = knn.predict(X_test)
tree_y = tree.predict(X_test)
lr_y = lr.predict(X_test)
svm_y = svm.predict(X_test)

f_scores_test = [f1_score(y_test, y_pred) for y_pred in [knn_y, tree_y, lr_y, svm_y]]

In [47]:
comparison = pd.DataFrame({'CV Train score': best_score_test,
                         'Test score': f_scores_test},
                         index = ['KNN', 'Tree', 'LR', 'SVM'])

In [48]:
comparison

,CV Train score,Test score
KNN,0.410593,0.412414
Tree,0.591211,0.579583
LR,0.502061,0.483981
SVM,0.500197,0.485660
